In [ ]:
import os

home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

In [ ]:
conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs
http://localhost:6006/

In [ ]:
import gym
from env import Env

env = gym.make(Env.MOUNTAINCAR.value)
print(dir(env))
print(f'{env._max_episode_steps=}')
print(f'{env.action_space=}')
print(f'{env.metadata=}')
print(f'{env.observation_space.shape[0]=}')
print(f'{env.reward_range=}')
print(f'{env.seed=}')
print(f'{env.spec=}')

In [1]:
# cart-pole 모음

from env import Env
from runner import RunnerParams

#dqn
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner
algo_param = DQNParams(buffer_limit=50000, n_train_start=4000,
                        n_node=128, start_epsilon=0.1, learning_rate=0.0001,
                        update_interval=40)

runner_param = RunnerParams(save_net=True, name_postfix=str(algo_param),
                            target_score=500.0,
                            interval=8, 
                            max_video=100, video_record_interval=200,
                            reward_scale=100.0)
DQNRunner(Env.CARTPOLE.value, algo_param, runner_param).run()

# ddqn
# from algorithms.ddqn import DDQNParams
# from algorithms.ddqn_runner import DDQNRunner
# runner_param = RunnerParams(target_score=500, reward_scale=100.0, max_video=100)
# DDQNRunner(Env.CARTPOLE.value, DDQNParams(), runner_param).run()

# ActorCritic
# from algorithms.actorcritic import ActorCriticParams
# from algorithms.actorcritic_runner import ActorCriticRunner
# runner_param = RunnerParams(target_score=500, reward_scale=100.0, max_video=100)
# ActorCriticRunner(Env.CARTPOLE.value, ActorCriticParams(), runner_param).run()

# PPO
# from algorithms.ppo import PPOParams
# from algorithms.ppo_runner import PPORunner

# algo_param = PPOParams()

# runner_param = RunnerParams(save_net=True, name_postfix=str(algo_param),
#                             target_score=500.0,
#                             interval=40, 
#                             max_video=100, video_record_interval=200,
#                             reward_scale=100.0)

# PPORunner(Env.CARTPOLE.value, algo_param, runner_param).run()

# PPOlstm
# from algorithms.ppolstm import PPOlstmParams
# from algorithms.ppolstm_runner import PPOlstmRunner
# runner_param = RunnerParams(save_net=True, target_score=500, reward_scale=100.0, max_video=100)
# PPOlstmRunner(Env.CARTPOLE.value, PPOlstmParams(), runner_param).run()

초기 설정
algorithm: DQN
env: CartPole-v1
state space: (4,)
action space: Discrete(2)
시뮬레이션 시작


C:\Users\kuro1\anaconda3\envs\py38-pytorch-gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


n_epi=0, avg_score=9.0 비디오 저장
에피소드: 0, 평균 점수: 9.0
n_buffer : 9, eps : 10.0%
에피소드: 8, 평균 점수: 9.0
n_buffer : 91, eps : 10.0%
에피소드: 16, 평균 점수: 10.0
n_buffer : 172, eps : 9.9%
에피소드: 24, 평균 점수: 9.0
n_buffer : 246, eps : 9.9%
에피소드: 32, 평균 점수: 9.0
n_buffer : 329, eps : 9.8%
에피소드: 40, 평균 점수: 11.0
n_buffer : 406, eps : 9.8%
에피소드: 48, 평균 점수: 10.0
n_buffer : 486, eps : 9.8%
에피소드: 56, 평균 점수: 10.0
n_buffer : 563, eps : 9.7%
에피소드: 64, 평균 점수: 11.0
n_buffer : 649, eps : 9.7%
에피소드: 72, 평균 점수: 12.0
n_buffer : 730, eps : 9.6%
에피소드: 80, 평균 점수: 9.0
n_buffer : 811, eps : 9.6%
에피소드: 88, 평균 점수: 11.0
n_buffer : 887, eps : 9.6%
에피소드: 96, 평균 점수: 9.0
n_buffer : 966, eps : 9.5%
에피소드: 104, 평균 점수: 10.0
n_buffer : 1044, eps : 9.5%
에피소드: 112, 평균 점수: 9.0
n_buffer : 1122, eps : 9.4%
에피소드: 120, 평균 점수: 10.0
n_buffer : 1208, eps : 9.4%
에피소드: 128, 평균 점수: 10.0
n_buffer : 1286, eps : 9.4%
에피소드: 136, 평균 점수: 10.0
n_buffer : 1363, eps : 9.3%
에피소드: 144, 평균 점수: 11.0
n_buffer : 1448, eps : 9.3%
에피소드: 152, 평균 점수: 8.0
n_buffer : 1526

In [1]:
# 모델 load
from env import Env
from runner import RunnerParams

# dqn
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner

algo_param = DQNParams(buffer_limit=50000, n_train_start=4000,
                        n_node=128, start_epsilon=0.1, learning_rate=0.0001,
                        update_interval=40)
runner_param = RunnerParams(train=False,
                            load_net=True, 
                            load_name='DQN-CartPole-v1-500.0-train=True-intvl=8-rwdscl=100.0-node=128-lRate=0.0001-gma=0.98-nBuf=50000-nBat=32-nStrt=4000-updIntvl=40-1632155680.pt',
                            name_postfix=str(algo_param),
                            target_score=999.0,
                            max_video=100, 
                            interval=1, video_record_interval=1,
                            reward_scale=100.0)

DQNRunner(Env.CARTPOLE.value, algo_param, runner_param).run()

# ddqn
# from algorithms.ddqn import DDQNParams
# from algorithms.ddqn_runner import DDQNRunner
# runner_param = RunnerParams(target_score=500, reward_scale=100.0, max_video=100)
# DDQNRunner(Env.CARTPOLE.value, DDQNParams(), runner_param).run()

# ActorCritic
# from algorithms.actorcritic import ActorCriticParams
# from algorithms.actorcritic_runner import ActorCriticRunner
# runner_param = RunnerParams(target_score=500, reward_scale=100.0, max_video=100)
# ActorCriticRunner(Env.CARTPOLE.value, ActorCriticParams(), runner_param).run()

# PPO
# from algorithms.ppo import PPOParams
# from algorithms.ppo_runner import PPORunner

# algo_param = PPOParams()
# runner_param = RunnerParams(train=False, load_net=True, load_name='PPO-CartPole-v1-500.0-train=True-intvl=40-rwdscl=100.0-node=128-lRate=0.0001-gma=0.98-lmb=0.95-epsclp=0.1-k=3-t=20-1632072908.pt',
#                             name_postfix=str(algo_param),
#                             target_score=999.0,
#                             max_video=100, 
#                             interval=1, video_record_interval=1,
#                             reward_scale=100.0)

# PPORunner(Env.CARTPOLE.value, algo_param, runner_param).run()

# PPOlstm
# from algorithms.ppolstm import PPOlstmParams
# from algorithms.ppolstm_runner import PPOlstmRunner
# runner_param = RunnerParams(save_net=True, target_score=500, reward_scale=100.0, max_video=100)
# PPOlstmRunner(Env.CARTPOLE.value, PPOlstmParams(), runner_param).run()

초기 설정
algorithm: DQN
env: CartPole-v1
state space: (4,)
action space: Discrete(2)
네트워크 불러오기
시뮬레이션 시작


C:\Users\kuro1\anaconda3\envs\py38-pytorch-gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


n_epi=0, avg_score=486.0 비디오 저장
에피소드: 0, 평균 점수: 486.0
n_buffer : 0, eps : 0.0%
에피소드: 1, 평균 점수: 387.0
n_buffer : 0, eps : 0.0%
에피소드: 2, 평균 점수: 439.0
n_buffer : 0, eps : 0.0%
에피소드: 3, 평균 점수: 498.0
n_buffer : 0, eps : 0.0%
에피소드: 4, 평균 점수: 402.0
n_buffer : 0, eps : 0.0%
에피소드: 5, 평균 점수: 473.0
n_buffer : 0, eps : 0.0%
에피소드: 6, 평균 점수: 409.0
n_buffer : 0, eps : 0.0%
에피소드: 7, 평균 점수: 380.0
n_buffer : 0, eps : 0.0%
에피소드: 8, 평균 점수: 380.0
n_buffer : 0, eps : 0.0%
에피소드: 9, 평균 점수: 399.0
n_buffer : 0, eps : 0.0%
에피소드: 10, 평균 점수: 472.0
n_buffer : 0, eps : 0.0%
에피소드: 11, 평균 점수: 416.0
n_buffer : 0, eps : 0.0%
에피소드: 12, 평균 점수: 391.0
n_buffer : 0, eps : 0.0%
에피소드: 13, 평균 점수: 427.0
n_buffer : 0, eps : 0.0%
에피소드: 14, 평균 점수: 431.0
n_buffer : 0, eps : 0.0%
에피소드: 15, 평균 점수: 427.0
n_buffer : 0, eps : 0.0%
에피소드: 16, 평균 점수: 456.0
n_buffer : 0, eps : 0.0%
에피소드: 17, 평균 점수: 500.0
n_buffer : 0, eps : 0.0%
종료 조건 만족. 최종 1번 평균 점수 500.0
시뮬레이션 종료


In [ ]:
## dqn cart-pole 학습

from env import Env
from runner import RunnerParams
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner

runner_param = RunnerParams(reward_scale=100.0, max_video=30)
algo_param = DQNParams()
DQNRunner(Env.CARTPOLE.value, algo_param, runner_param).run()

In [ ]:
## ddqn lunar-lander 불러오기

from env import Env
from runner import RunnerParams
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner

runner_param = RunnerParams(load_net=True, load_name='node512-score165-DQN-LunarLander-v2-1631737560.pt',
                            train=False,
                            target_score=150, 
                            reward_scale=30.0, max_video=100, video_record_interval=1,
                            step_wrapper=lambda x: (x[0], x[1], x[2], x[3]))
algo_param = DQNParams(n_node=512, batch_size=32, buffer_limit=100000, 
                        n_train_start=8000, start_epsilon=0.2,
                        update_interval=40)
DQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

In [ ]:
## ddqn lunar-lander
from env import Env
from runner import RunnerParams
from algorithms.ddqn import DDQNParams
from algorithms.ddqn_runner import DDQNRunner

runner_param = RunnerParams(save_net=True,
                            train=True,
                            target_score=200, 
                            reward_scale=30.0, max_video=100, video_record_interval=200,
                            step_wrapper=lambda x: x)
algo_param = DDQNParams(n_node=512, batch_size=64, buffer_limit=50000, 
                        n_train_start=4000, start_epsilon=0.2,
                        update_interval=20)
DDQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

In [ ]:
## ddqn lunar-lander 불러오기

from env import Env
from runner import RunnerParams
from algorithms.ddqn import DDQNParams
from algorithms.ddqn_runner import DDQNRunner

runner_param = RunnerParams(save_net=False, load_net=True, load_name='node512-score213-DDQN-LunarLander-v2-1631750946.pt',
                            train=False,
                            target_score=200, 
                            reward_scale=30.0, max_video=100, video_record_interval=1,
                            step_wrapper=lambda x: x)
algo_param = DDQNParams(n_node=512, batch_size=64, buffer_limit=50000, 
                        n_train_start=4000, start_epsilon=0.2,
                        update_interval=20)
DDQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

In [ ]:
# 알고리즘 테스트 

# ppo

# from tester import RunnerTester
# from algorithms.ppo_runner import PPORunner
# from algorithms.ppo import PPOParams
# from env import Env
# algo_params = PPOParams()
# result = RunnerTester(PPORunner, algo_params, [Env.MOUNTAINCAR]).test()
# print('통과' if result else '실패')


# ppolstm

# from tester import RunnerTester
# from algorithms.ppolstm_runner import PPOlstmRunner
# from algorithms.ppolstm import PPOlstmParams
# from env import Env
# algo_params = PPOlstmParams()
# result = RunnerTester(PPOlstmRunner, algo_params, [Env.MOUNTAINCAR]).test()
# print('통과' if result else '실패')


# actorcritic

# from tester import RunnerTester
# from algorithms.actorcritic_runner import ActorCriticRunner
# from algorithms.actorcritic import ActorCriticParams
# from env import Env
# algo_params = ActorCriticParams()
# result = RunnerTester(ActorCriticRunner, algo_params, [Env.CARTPOLE]).test()
# print('통과' if result else '실패')



# dqn
# from tester import RunnerTester
# from algorithms.dqn_runner import DQNRunner
# from algorithms.dqn import DQNParams
# from env import Env
# algo_params = DQNParams()
# result = RunnerTester(DQNRunner, algo_params, [Env.CARTPOLE]).test()
# print('통과' if result else '실패')



# ddqn
from tester import RunnerTester
from algorithms.ddqn_runner import DDQNRunner
from algorithms.ddqn import DDQNParams
from env import Env
algo_params = DDQNParams(start_epsilon=0.2, n_node=128, n_train_start=20000, 
                        buffer_limit=100000)
result = RunnerTester(DDQNRunner, algo_params, [Env.CARTPOLE]).test()
print('통과' if result else '실패')